____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno sabado mañana__<br/>
__Elaborado por: Nicolas Aguirre__
____

# Clase_03: Analisis Exploratorio de los Datos
## Airbnb & Google Colaboratory

# Librerias

In [ ]:
#Importar paquetes de herramientas:
#Datos
import pandas as pd
import numpy as np
#Graficos 
import matplotlib.pyplot as plt
import seaborn as sns
#Otros 
import warnings
warnings.filterwarnings('ignore')

# Google Drive

Cuando trabajamos en el entorno de Google Colaboratory, tenemos a disposicion una Virtual Machine (VM). El iniciar la VM no tenemos acceso a nuestro Google Drive, por lo cual debemos primero que nada 'montar' nuestros archivos.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Una buena practica es definir los 'path' o direcciones hacia la carpeta en la cual estamos trabajando en variables.

In [ ]:
# Direccion root donde está la jupyter-notebook
root_path = "/content/gdrive/My Drive/Colab Notebooks/ClusterAI/Airbnb/"
# Direccion donde guardaremos las imagenes
plot_path = root_path + "figures/" 

# Dataset

El dataset  que usaremos se encuentra en:
  
https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

* Descargar e importarlo


* Verificar que se haya cargado bien el dataset


* Dimension del dataset

In [ ]:
df = pd.read_csv(root_path+"AB_NYC_2019.csv",index_col="id")

In [ ]:
print(f'Shape: {np.shape(df)}')
df.head(3)

# Limpieza

* **Duplicados**


* **Cantidad** y **% de NaN** por columna


* Luego de quitarlos, devolver shape del df

In [ ]:
df.drop_duplicates(inplace=True,keep='first')

In [ ]:
cant_NaN = df.isnull().sum()
print(cant_NaN,'\r\n')

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

In [ ]:
#Limpiamos registros
df.dropna(how ='any', inplace = True)

In [ ]:
# Verificamos como quedo el data-set
print(f'Nos quedamos con un df de {df.shape[0]} filas x{df.shape[1]} columnas')

# Practica!

1. Crear columna $\log(precio)$. Grafico de la distribucion de precio y de $log(precio)$


2. En base al que mejor se ajuste, comparar los precios segun los distintos *neighbourhood_group* (boxplot)


3. Mediana, Limite inferior y superior del *neighbourhood_group* con mayor y menor precio.


4. Crear una nueva columa llamada *Brand*, con 7 categorias de precios. 
        
    Bandas = [0, 50, 75, 100, 150, 200, 250]


5. Boxplot de *Brand* vs $\log(Reviews)$


6. Scatter plot de Longitud y Latitud donde se distinga *neighbourhood_group*


7. Scatter plot de Longitud y Latitud donde se distingan las brands


8. Cantidad de barrios


9. Comparacion de la cantidad de hospedajes para los 10 barrios mas populares (barras)


10. Cantidad de hospedajes para los 10 barrios mas populares, segregados por brands (pivot_table)


## Log de precio y dist. estimada

In [ ]:
#Columna log precios
df['log_price'] = np.log(1+df['price'])

#Plot Precio: Vamos a crear esta vez un 'plot' que contenga dos figuras.
# Formalmente, lo que crearemos sera una unica figura, divivida en
# filas x columna = 1 ,2
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2,
                               figsize=(15,5))

# Seaborn, al ser una libreria construida sobre Matplotlib, permite
# que le pasemos el argumento "ax=".
sns.kdeplot(df.price, color="b", shade = True,ax=ax1)
ax1.set_xlabel("Price")
# Este es el titulo del axis numero 2
ax1.set_title("KDE")

#Plot Log-Precio
sns.distplot(np.log(1+df['price']),color='b',ax=ax2)
ax2.set_xlabel("Price")

# Este es el titulo del axis numero 2
ax2.set_title("Log-KDE")

# Mientras que el titulo 'general', lo asignamos
# a la figure.
fig.suptitle('Distribucion de Precio',size = 20)

#####################
# Guardar Plots!
#####################
# Siempre debemos guardar el plot ANTES del mostrarlo (plt.show())
# Nombre del plot
plot_name = str(plot_path+'dist_price')
plot_format = ".png"
plt.savefig(plot_name+plot_format)

# Por ultimo, mostramos todo lo que hayamos ploteado hasta el momento
plt.show()

## Box plot *neighbourhood_group*

In [ ]:
# Comparar los precios segun los distintos "neighbourhood_group" (boxplot)
g = sns.catplot(x='neighbourhood_group',y='log_price',data=df,
                kind="box", height = 5 ,aspect=3, palette = "muted")
g.despine(left=True) # Para quitar la linea del eje Y
g.set_xticklabels(rotation=45) # Rotamos los labels del eje X
plt.xlabel("Barrios",size = 18)
plt.ylabel("Log_Precio",size = 18)
plt.title('Plot de log_price vs Barrio',size = 24)

# esto ajusta la imagen que se va a guardar a los bordes del
# la figure
plt.tight_layout()

plot_name = str(plot_path+'log_price_barrio')
plot_format = ".png"
plt.savefig(plot_name+plot_format)
plt.show()

## Mediana, Lim. Inferior y Sup.

### enumrate( zip() )

In [ ]:
list_x = ['a','b','c','d','e']
list_y = [1,2,3,4,5]
list_z = ['a',2,'c',4,'e']
# La funcion zip() empaqueta objetos y los va devolviendo de a conjuntos.
for (x_i,y_i,z_i) in zip(list_x,list_y,list_z):
  print('lista_x:',x_i,'lista_y:',y_i,'lista_z:',z_i)

In [ ]:
# La fucion enumrate() nos devuelve un "objeto iterable" que devuelve
# indices + "datos"
# Combinando estas dos funciones vamos a obtener:
for i, (x_i,y_i,z_i) in enumerate(zip(list_x,list_y,list_z)):
  print('Indice:',i,'lista_x:',x_i,'lista_y:',y_i,'lista_z:',z_i)

In [ ]:
# Mediana, Limite inferior y superior del 'neighbourhood_group'
# con mayor y menor precio.

# Creemos un DF vacio con el nombre de columnas asignado donde guardaremos
# resultados
column_names = ["Mediana", "Lim. Inferior", "Lim. Superior"]
df2delet = pd.DataFrame(columns = column_names)

# Barrios sobre los que queremos "iterar"
barrio_list = ["Manhattan", "Bronx"]
# Creemos subplots y veamos que es lo que nos esta devolviendo
fig, axs = plt.subplots(nrows=1, ncols=2,figsize=(10,5))
print("El objeto type de axs es: {}, y tiene un shape de {}".format(
    type(axs),np.shape(axs)))

# Vamos a iterar sobre cada Axis y barrio_list
for i ,(ax_i,barrio_i) in enumerate(zip(axs,barrio_list)):
  print(i,type(ax_i),barrio_i)
  df_pivot= df[df['neighbourhood_group']==barrio_i]
  A = ax_i.boxplot(df_pivot['log_price'])
  ax1.set_title(barrio_i)
  #Med.
  medians = [median.get_ydata() for median in A["medians"]]
  #whiskers
  whiskers = [whiskers.get_ydata() for whiskers in A["whiskers"]]
  # df que vamos agregar a df2delet
  df_i = pd.DataFrame({'Mediana': medians[0][0],
                              'Lim. Inferior': whiskers[0][0],
                              'Lim. Superior': whiskers[1][1]},index=[barrio_i])
  # agregamos df
  df2delet = df2delet.append(df_i)

plt.show()

df2delet

## Brands

In [ ]:
#Definimos y creamos las bandas:

df.loc[(df['price'] >= 0) & (df['price'] <= 50), 'Banda'] = 1
df.loc[(df['price'] > 50) & (df['price'] <= 75), 'Banda']   = 2
df.loc[(df['price'] > 75) & (df['price'] <= 100), 'Banda']   = 3
df.loc[(df['price'] > 100) & (df['price'] <= 150), 'Banda']   = 4
df.loc[(df['price'] > 150) & (df['price'] <= 200), 'Banda']   = 5
df.loc[(df['price'] > 200) & (df['price'] <= 250), 'Banda']   = 6
df.loc[(df['price'] > 250), 'Banda']  = 7
df['Banda'] = df['Banda'].astype(int)

## Brand vs $\log(Reviews)$

In [ ]:
#Boxplot de "Banda" vs log(Reviews)
df['log_review'] = np.log(1+df['number_of_reviews'])

g = sns.catplot(x="Banda",y='log_review',data=df, kind="box", height = 5,aspect=3 ,palette = "muted")
g.despine(left=True)
g.set_xticklabels(rotation=90)
g = g.set_ylabels("log_Reviews")
plt.title('Boxen plot Bandas VS Reviews',size = 20)
plt.tight_layout()
plot_name = str(plot_path+'brands_reviews')
plot_format = ".png"
plt.savefig(plot_name+plot_format)
plt.show()

## Scatter Plot: *neighbourhood_group*

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(df['longitude'],df['latitude'],hue=df['neighbourhood_group'],palette='muted')
plt.show()

## Scatter Plot: Brands

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(df['longitude'],df['latitude'],hue=df['Banda'], # Datos
                edgecolor= "none", # color de los bordes de los puntos 
                alpha=0.5, # transparencia
                palette='viridis') # mapa de colores
plt.show()

## Qty de barrios

In [ ]:
q_barrios = len(df['neighbourhood'].unique())
print(f'En total hay {q_barrios} barrios\r\n')

## Hospedajes para los 10 barrios mas populares

In [ ]:
top_neighbourhood = df['neighbourhood'].value_counts().index[0:10]


plt.figure(figsize=(15,5))
g = sns.countplot(x='neighbourhood',data=df, palette = "muted",order=top_neighbourhood)
g.set_xticklabels(g.get_xticklabels(), rotation=45, ha="right")
plt.title('Cuenta por Barrio',size = 20)
plt.xlabel("Barrio")
plt.ylabel("Count")
plt.tight_layout()
plot_name = str(plot_path+'top_neighbourhood')
plot_format = ".png"
plt.savefig(plot_name+plot_format)
plt.show()

## Cantidad de hospedajes para los 10 barrios mas populares, segregados por brands 

### Pivot Table

In [ ]:
df2delete = pd.DataFrame({
    "A": ["foo", "foo", "foo", "foo", "foo","bar", "bar", "bar", "bar"],
    "B": ["one", "one", "one", "two", "two","one", "one", "two", "two"],
    "C": ["small", "large", "large", "small","small", "large", "small", "small","large"],
    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
    "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]
    })
df2delete

In [ ]:
table = pd.pivot_table(df2delete, #Datos
                       values=['D'], #valores que van a aparecer dentro de la tabla
                       index=['A', 'B'], #Los indices que ve a tener
                       columns=['C'], # La columna que va a tener
                       aggfunc=np.sum) # Que se aplicara a los valores
table

In [ ]:
table = pd.pivot_table(df2delete,
                       # Multiples valores
                       values=['D', 'E'],
                       index=['A'],
                       # Tambien por columna
                       columns=['C'],
                       # Y con multiples funciones para cada uno de los valores
                       aggfunc={'D': np.mean,
                                'E': [min, max, np.mean]})
table

In [ ]:
table = pd.pivot_table(
    # Datframe
    df[df['neighbourhood'].isin(top_neighbourhood)],
    # Filas
    index='neighbourhood',
    # Columnas
    columns = 'Banda',
    # Quiero ver los precios
    values='price' ,
    # Si hay NaN, pongo un cero 
    fill_value=0,
    # Que funcion aplico?
    aggfunc = 'count')
table

# Consultas?